In [ ]:
# use exact versions of these in order to preserve RANK ordering better
requirements = "numpy==1.26.4 pandas==2.2.2 scikit-learn==1.5.1 xgboost==2.1.0 lightgbm==4.5.0 catboost==1.2.5 aplr==10.6.1"
!pip install -U --quiet {requirements}

In [ ]:
# install interpret if not already installed
try:
    import interpret
except ModuleNotFoundError:
    !pip install -U --quiet interpret-core

In [ ]:
# install powerlift if not already installed
try:
    import powerlift
except ModuleNotFoundError:
    !pip install -U --quiet powerlift[datasets,postgres]

In [ ]:
def trial_filter(task):
    min_samples = 1
    max_samples = 1000000000000
    min_features = 1
    max_features = 1000000000000
    if task.scalar_measure("n_rows") < min_samples:
        return []
    if max_samples < task.scalar_measure("n_rows"):
        return []
    if task.scalar_measure("n_cols") < min_features:
        return []
    if max_features < task.scalar_measure("n_cols"):
        return []

    
    if task.origin == "openml_automl_regression":
        pass  # include in benchmark
    elif task.origin == "openml_automl_classification":
        return []
    elif task.origin == "openml_cc18":
        pass  # include in benchmark
    elif task.origin == "pmlb":
        if task.problem == "binary":
            return []
        elif task.problem == "multiclass":
            return []
        elif task.problem == "regression":
            return []
        else:
            raise Exception(f"Unrecognized problem {task.problem}")
    else:
        raise Exception(f"Unrecognized origin {task.origin}")

    
    exclude_set = set()
#    exclude_set = set(['isolet', 'Devnagari-Script', 'CIFAR_10', 'Airlines_DepDelay_10M'])
#    exclude_set = set([
#        'Fashion-MNIST', 'mfeat-pixel', 'Bioresponse',
#        'mfeat-factors', 'isolet', 'cnae-9', "Internet-Advertisements",
#        'har', 'Devnagari-Script', 'mnist_784', 'CIFAR_10',
#        'Airlines_DepDelay_10M',
#    ])
    if task.name in exclude_set:
        return []


    # exclude duplicates of a dataset if they appear twice
    global global_duplicates
    try:
        duplicates = global_duplicates
    except NameError:
        duplicates = set()
        global_duplicates = duplicates
    key = (task.name, task.scalar_measure("n_rows"), task.scalar_measure("n_cols"))
    if key in duplicates:
        print(f"Excluding duplicate: {key}")
        return []
    else:
        duplicates.add(key)


    return [
        "ebm",
        "xgb",
        "lgbm",
        "catboost",
        "rf_xgb",
        "rf_sk",
        "linear_sv",
        # "aplr",
    ]

In [ ]:
def trial_runner(trial):
    seed=42 + int(trial.replicate_num)

    from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor
    from xgboost import XGBClassifier, XGBRegressor, XGBRFClassifier, XGBRFRegressor
    from lightgbm import LGBMClassifier, LGBMRegressor
    from catboost import CatBoostClassifier, CatBoostRegressor
    from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
    from sklearn.svm import LinearSVC, LinearSVR
    from sklearn.calibration import CalibratedClassifierCV
    from aplr import APLRClassifier, APLRRegressor
    from sklearn.metrics import roc_auc_score, root_mean_squared_error, log_loss
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.compose import ColumnTransformer
    from sklearn.impute import SimpleImputer
    from sklearn.pipeline import Pipeline
    import numpy as np
    from time import time
    import warnings

    X, y, meta = trial.task.data(["X", "y", "meta"])

    for col in X.columns:
        # catboost doesn't like missing categoricals, so make them a category
        col_data = X[col]
        if str(col_data.dtype) == "category" and col_data.isnull().any():
            X[col] = col_data.cat.add_categories('nan').fillna('nan')
    
    cat_bools = meta["categorical_mask"]
    cat_cols = [i for i, val in enumerate(cat_bools) if val]
    num_cols = [i for i, val in enumerate(cat_bools) if not val]
    
    stratification = None
    if trial.task.problem in ["binary", "multiclass"]:
        # stratification = y
        pass  # Re-enable stratification if dataset fails from absent class in train/test sets (PMLB)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=stratification, random_state=seed)

    # Build optional preprocessor for use by methods below
    # missing categoricals already handled above by making new "nan" category
    cat_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.int16)
    num_imputer = SimpleImputer(strategy="mean")
    transformers = [("cat", cat_encoder, cat_cols), ("num", num_imputer, num_cols)]
    ct = ColumnTransformer(transformers=transformers, sparse_threshold=1.0)  # densify or sparsify


    ebm_params = {}
    xgb_params = {}
    lgbm_params = {}
    catboost_params = {}
    rf_xgb_params = {}
    rf_sk_params = {}
    linear_sv_params = {}
    aplr_params = {}

    ebm_params["feature_types"] = ["nominal" if cat else "continuous" for cat in cat_bools]
    ebm_params["n_jobs"] = -1
    xgb_params["enable_categorical"] = True
    xgb_params["feature_types"] = ["c" if cat else "q" for cat in cat_bools]
    lgbm_params["verbosity"] = -1
    catboost_params["verbose"] = False
    rf_xgb_params["enable_categorical"] = True
    rf_xgb_params["feature_types"] = ["c" if cat else "q" for cat in cat_bools]
    rf_sk_params["n_jobs"] = -1
    aplr_params["m"] = 3000

    if 1000 < trial.task.scalar_measure("n_cols"):
        # TODO: EBMs can crash for now with too many interactions, so limit it until we have better fix
        ebm_params["interactions"] = 0

    # DEBUG params to make the algorithms super fast
    #if 10000 < len(y_train):
    #    debug_stratify = y_train if trial.task.problem in ["binary", "multiclass"] else None
    #    X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=len(y_train) - 5000, stratify=debug_stratify, random_state=seed)
    #ebm_params["max_rounds"] = 1
    #ebm_params["interactions"] = 0
    #xgb_params["n_estimators"] = 1
    #lgbm_params["n_estimators"] = 1
    #catboost_params["n_estimators"] = 1
    #rf_xgb_params["n_estimators"] = 1
    #rf_sk_params["n_estimators"] = 1
    #aplr_params["m"] = 1
    
    # Specify method
    if trial.task.problem in ["binary", "multiclass"]:
        fit_params = {"X":X_train, "y":y_train}
        if trial.method.name == "ebm":
            est = ExplainableBoostingClassifier(**ebm_params)
        elif trial.method.name == "xgb":
            est = XGBClassifier(**xgb_params)
            fit_params["verbose"] = False
        elif trial.method.name == "lgbm":
            est = LGBMClassifier(**lgbm_params)
            fit_params["categorical_feature"] = cat_cols
        elif trial.method.name == "catboost":
            est = CatBoostClassifier(**catboost_params)
            fit_params["cat_features"] = cat_cols
        elif trial.method.name == "rf_xgb":
            est = XGBRFClassifier(**rf_xgb_params)
            fit_params["verbose"] = False
        elif trial.method.name == "rf_sk":
            est = Pipeline([("ct", ct), ("est", RandomForestClassifier(**rf_sk_params))])
        elif trial.method.name == "linear_sv":
            if trial.task.problem == "multiclass":
                est = Pipeline([("ct", ct), ("est", CalibratedClassifierCV(OneVsRestClassifier(LinearSVC(**linear_sv_params), n_jobs=-1)))])
            else:
                est = Pipeline([("ct", ct), ("est", CalibratedClassifierCV(LinearSVC(**linear_sv_params)))])
        elif trial.method.name == "aplr":
            est = Pipeline(
                [
                    ("ct", ct),
                    (
                        "est",
                        APLRClassifier(**aplr_params),
                    ),
                ]
            )
            y_train = y_train.astype(str).to_numpy()
            y_test = y_test.astype(str).to_numpy()
            fit_params = {"X":X_train, "y":y_train}
        else:
            raise Exception(f"Unrecognized method name {trial.method.name}")

        predict_fn = est.predict_proba
    elif trial.task.problem == "regression":
        fit_params = {"X":X_train, "y":y_train}
        if trial.method.name == "ebm":
            est = ExplainableBoostingRegressor(**ebm_params)
        elif trial.method.name == "xgb":
            est = XGBRegressor(**xgb_params)
            fit_params["verbose"] = False
        elif trial.method.name == "lgbm":
            est = LGBMRegressor(**lgbm_params)
            fit_params["categorical_feature"] = cat_cols
        elif trial.method.name == "catboost":
            est = CatBoostRegressor(**catboost_params)
            fit_params["cat_features"] = cat_cols
        elif trial.method.name == "rf_xgb":
            est = XGBRFRegressor(**rf_xgb_params)
            fit_params["verbose"] = False
        elif trial.method.name == "rf_sk":
            est = Pipeline([("ct", ct), ("est", RandomForestRegressor(**rf_sk_params))])
        elif trial.method.name == "linear_sv":
            est = Pipeline([("ct", ct), ("est", LinearSVR(**linear_sv_params))])
        elif trial.method.name == "aplr":
            est = Pipeline(
                [
                    ("ct", ct),
                    (
                        "est",
                        APLRRegressor(**aplr_params),
                    ),
                ]
            )
            y_train = y_train.astype(str).to_numpy()
            y_test = y_test.astype(str).to_numpy()
            fit_params = {"X":X_train, "y":y_train}
        else:
            raise Exception(f"Unrecognized method name {trial.method.name}")

        predict_fn = est.predict
    else:
        raise Exception(f"Unrecognized problem {trial.task.problem}")

    global global_counter
    try:
        global_counter += 1
    except NameError:
        global_counter = 0
    
    # Train
    print(f"FIT: {global_counter}, {trial.task.origin}, {trial.task.name}, {trial.method.name}, classes:{int(trial.task.scalar_measure('n_classes'))}, features:{int(trial.task.scalar_measure('n_cols'))}, samples:{int(trial.task.scalar_measure('n_rows'))}")
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        start_time = time()
        est.fit(**fit_params)
        elapsed_time = time() - start_time
    trial.log("fit_time", elapsed_time)
    
    # Predict
    start_time = time()
    predictions = predict_fn(X_test)
    elapsed_time = time() - start_time
    trial.log("predict_time", elapsed_time)

    if trial.task.problem == "binary":
        predictions = predictions[:,1]

        eval_score = roc_auc_score(y_test, predictions)
        trial.log("auc", eval_score)

        eval_score2 = log_loss(y_test, predictions)
        trial.log("log_loss", eval_score2)
    elif trial.task.problem == "multiclass":
        eval_score = roc_auc_score(y_test, predictions, average="weighted", multi_class="ovo")
        trial.log("multi_auc", eval_score)

        eval_score2 = log_loss(y_test, predictions)
        trial.log("cross_entropy", eval_score2)
    elif trial.task.problem == "regression":
        # Use NRMSE-IQR (normalized root mean square error by the interquartile range)
        # so that datasets with large predicted values do not dominate the benchmark
        # and the range is not sensitive to outliers. The rank is identical to RMSE.
        # https://en.wikipedia.org/wiki/Root_mean_square_deviation

        # Get quartile_range from the full dataset for consistency across seeds.
        q75, q25 = np.percentile(y, [75, 25])
        interquartile_range = q75 - q25

        eval_score = root_mean_squared_error(y_test, predictions) / interquartile_range
        trial.log("nrmse", eval_score)
    else:
        raise Exception(f"Unrecognized problem {trial.task.problem}")

    print(eval_score)

In [ ]:
import datetime
experiment_name = datetime.datetime.now().strftime("%Y_%m_%d_%H%M__") + "myexperiment"
# experiment_name = "yyyy_mm_dd_hhmm__myexperiment"

print("Experiment name: " + experiment_name)

In [ ]:
force_recreate=False
exist_ok=True
is_local=True
n_replicates=10

import os
if is_local:
    conn_str = f"sqlite:///{os.getcwd()}/powerlift.db"
else:
    from azure.identity import AzureCliCredential
    credential = AzureCliCredential()
    
    from dotenv import load_dotenv
    load_dotenv()
    TIMEOUT_SEC = 60 * 60 * 24 * 180  # 180 days
    wheel_filepaths = ["interpret_core-0.6.3-py3-none-any.whl", "powerlift-0.1.11-py3-none-any.whl"]
    n_containers=198
    conn_str = os.getenv("DOCKER_DB_URL")
    azure_tenant_id = os.getenv("AZURE_TENANT_ID")
    azure_client_id = os.getenv("AZURE_CLIENT_ID")
    azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")
    subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
    resource_group = os.getenv("AZURE_RESOURCE_GROUP")

from powerlift.bench import retrieve_openml_automl_regression, retrieve_openml_automl_classification, retrieve_openml_cc18, retrieve_catboost_50k, retrieve_pmlb
from powerlift.bench import Benchmark, Store, populate_with_datasets
from powerlift.executors import LocalMachine, AzureContainerInstance
from itertools import chain

# Initialize database (if needed).
store = Store(conn_str, force_recreate=force_recreate, print_exceptions=True)

benchmark = Benchmark(store, name=experiment_name)

In [ ]:
cache_dir="~/.powerlift"
data_retrieval = chain(
    retrieve_openml_automl_regression(cache_dir=cache_dir),
    # retrieve_openml_automl_classification(cache_dir=cache_dir),
    retrieve_openml_cc18(cache_dir=cache_dir),
    # retrieve_catboost_50k(cache_dir=cache_dir),
    # retrieve_pmlb(cache_dir=cache_dir),
)

# This downloads datasets once and feeds into the database.
populate_with_datasets(store, data_retrieval, exist_ok=exist_ok)

if is_local:
    benchmark.run(trial_runner, trial_filter, n_replicates=n_replicates, executor=LocalMachine(store, debug_mode=True))
else:
    executor = AzureContainerInstance(
        store, azure_tenant_id, azure_client_id, azure_client_secret, subscription_id, resource_group, credential,
        image="mcr.microsoft.com/devcontainers/python:latest",
        pip_install= requirements + " psycopg2-binary",
        wheel_filepaths=wheel_filepaths,
        n_running_containers=n_containers, num_cores=4, mem_size_gb=16, delete_group_container_on_complete=True
    )
    benchmark.run(trial_runner, trial_filter, timeout=TIMEOUT_SEC, n_replicates=n_replicates, executor=executor)

In [ ]:
#benchmark.wait_until_complete()

In [ ]:
results_df = benchmark.results()
results_df['meta'] = results_df['meta'].apply(lambda x: str(x))
results_df = results_df.sort_values(by=['task', 'method', 'meta', 'replicate_num', 'name'])
results_df.to_csv(f"{experiment_name}.csv", index=None)

status_df = benchmark.status()
for errmsg in status_df["errmsg"]:
    if errmsg is not None:
        print("ERROR: " + str(errmsg))
print(status_df['status'].value_counts().to_string(index=True, header=False))

In [ ]:
import pandas as pd

# reload if analyzing later
results_df = pd.read_csv(f"{experiment_name}.csv")

averages = results_df.groupby(['method', 'name'])['num_val'].mean().unstack().reset_index()

metric_ranks = results_df.pivot_table('num_val', ['task', 'name'], ['method', 'replicate_num'])
metric_ranks = metric_ranks.rank(axis=1, ascending=True, method='min')
metric_ranks = metric_ranks.stack(level='replicate_num', future_stack=True)
metric_ranks = metric_ranks.groupby('name').mean().transpose()
metric_ranks.columns = [f"{col}_RANK" for col in metric_ranks.columns]
metric_ranks = metric_ranks.reset_index()

overall_rank = results_df[results_df['name'].isin(['log_loss', 'cross_entropy', 'nrmse'])]
overall_rank = overall_rank.pivot_table('num_val', 'task', ['method', 'replicate_num'])
overall_rank = overall_rank.rank(axis=1, ascending=True, method='min')
overall_rank = overall_rank.stack(level='replicate_num', future_stack=True)
overall_rank = overall_rank.mean()
overall_rank = overall_rank.to_frame(name='RANK').reset_index()

desired_columns = ['method', 'RANK', 'auc', 'multi_auc', 'nrmse', 'log_loss_RANK', 'cross_entropy_RANK', 'nrmse_RANK', 'fit_time', 'predict_time']
combined_df = averages.merge(metric_ranks, on='method').merge(overall_rank, on='method')
combined_df = combined_df.sort_values(by='RANK')
combined_df = combined_df.reindex(columns=desired_columns)

print(combined_df.to_string(index=False))

In [ ]:
desired_columns = ['method', 'RANK', 'auc', 'multi_auc', 'nrmse', 'log_loss', 'cross_entropy', 'fit_time', 'predict_time']
row_order = combined_df["method"]

counts = results_df.groupby(['method', 'name']).size().unstack()
counts = counts.reindex(row_order, axis=0).reset_index()
counts['RANK'] = 0
if 'log_loss' in counts.columns:
    counts['RANK'] += counts['log_loss']
if 'cross_entropy' in counts.columns:
    counts['RANK'] += counts['cross_entropy']
if 'nrmse' in counts.columns:
    counts['RANK'] += counts['nrmse']
counts = counts.reindex(columns=desired_columns)
print(counts.to_string(index=False))

In [ ]:
filtered_df = results_df[results_df['name'].isin(['log_loss', 'cross_entropy', 'nrmse'])]
grouped = filtered_df.groupby(['task', 'method', 'name'])['num_val'].agg(['mean']).reset_index()
pivot_table = grouped.pivot_table(index=['task', 'name'], columns=['method'], values=['mean'])
pivot_table.columns = pivot_table.columns.droplevel(0)
pivot_table = pivot_table.reindex(row_order, axis=1)
pivot_table['ratio'] = pivot_table['ebm'] / pivot_table['xgb']
pivot_table = pivot_table.sort_values(by='ratio')
task_order = pivot_table.index
pivot_table = pivot_table.reset_index()
print(pivot_table.to_string())

In [ ]:
filtered_df = results_df[results_df['name'].isin(['log_loss', 'cross_entropy', 'nrmse'])]
grouped = filtered_df.groupby(['task', 'method', 'name'])['num_val'].agg(['std']).reset_index()
pivot_table = grouped.pivot_table(index=['task', 'name'], columns=['method'], values=['std'])
pivot_table.columns = pivot_table.columns.droplevel(0)
pivot_table = pivot_table.reindex(row_order, axis=1)
pivot_table = pivot_table.reindex(task_order, axis=0).reset_index()
print(pivot_table.to_string())

In [ ]:
filtered_df = results_df[results_df['name'].isin(['log_loss', 'cross_entropy', 'nrmse'])]
grouped = filtered_df.groupby(['task', 'method', 'name'])['num_val'].agg(['count']).reset_index()
pivot_table = grouped.pivot_table(index=['task', 'name'], columns=['method'], values=['count'])
pivot_table = pivot_table.fillna(0).astype(int)
pivot_table.columns = pivot_table.columns.droplevel(0)
pivot_table = pivot_table.reindex(row_order, axis=1)
pivot_table = pivot_table.reindex(task_order, axis=0).reset_index()
print(pivot_table.to_string())

In [ ]:
fit_times = results_df[results_df['name'] == 'fit_time']
fit_times = fit_times.pivot_table('num_val', 'task', 'method')
fit_times = fit_times.dropna()
fit_times["ratios"] = fit_times['ebm'] / fit_times['xgb']
import numpy as np
fit_times_deciles = np.percentile(fit_times["ratios"], [90, 80, 70, 60, 50, 40, 30, 20, 10])
fit_times_deciles = [f"{decile:.2f}  " for decile in fit_times_deciles]
max_ratio= fit_times["ratios"].max()
min_ratio= fit_times["ratios"].min()
print("fit time ratio deciles:")
print(*fit_times_deciles)
print(f"max: {max_ratio:.2f}")
print(f"min: {min_ratio:.2f}")